In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense , LSTM , Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
dataM = pd.read_csv('Male-Names.csv')
dataF = pd.read_csv('Female-Names.csv')


In [3]:
df = pd.concat([dataM , dataF])

In [4]:
df.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [5]:

def should_keep(word):
    if(len(word) > 19):
        return False
    char_set  = [' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
    for ch in word: 
        if ch not in set(char_set):
            return False
    return True
   
     

In [6]:
def clean(word):
    name = str(word)
    name = name.lower()
    if should_keep(name):
        return name
    else:
        return None
df.name = df.name.apply(lambda word : clean(word) )

In [7]:
df = df.dropna()

In [8]:
df.head(-5)

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian
...,...,...,...
15368,priti,f,indian
15369,kavita,f,indian
15372,anjum,f,indian
15373,miss reena,f,indian


In [9]:
char_set  = [' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
char2idx = {}
index = 0
for ch in char_set:
    char2idx[ch] = index
    index+=1

In [10]:
df = df.drop('race' , axis = 1)
vector_len = 39
max_word_len = 20
words = []
labels = []
for name, gender in df.itertuples(index = False):
    one_hots_word = []
    for ch in name:
        vec = np.zeros(vector_len)
        vec[char2idx[ch]] = 1
        one_hots_word.append(vec)
    
    
    for _ in range(max_word_len - len(name)):
        vec = np.zeros(vector_len)
        vec[char2idx['END']] = 1
        one_hots_word.append(vec)
        
    one_hots_word = np.array(one_hots_word)
    words.append(one_hots_word)
    labels.append(gender)

In [11]:
labels[-20:]

['f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f']

In [12]:
words = np.array(words)

In [13]:
words.shape

(28000, 20, 39)

In [14]:
labels = np.array(labels)
ohe  = OneHotEncoder()

labels_one_hot = ohe.fit_transform(labels.reshape(-1 , 1)).todense()

In [15]:
labels_one_hot[:10]

matrix([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [16]:
model = Sequential()
model.add(LSTM(128 , input_shape = (20 , 39) , return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           86016     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 217,858
Trainable params: 217,858
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='rmsprop' , loss = 'categorical_crossentropy' , metrics = ['acc'] )

In [18]:
X_train, X_test, y_train, y_test = train_test_split(words,labels_one_hot , test_size=0.33, random_state=42)

In [19]:
model.fit(X_train , y_train , epochs=10)

Epoch 1/10
587/587 [==============================] - 9s 16ms/step - loss: 0.5197 - acc: 0.7450
Epoch 2/10
587/587 [==============================] - 9s 16ms/step - loss: 0.3822 - acc: 0.8388
Epoch 3/10
587/587 [==============================] - 9s 16ms/step - loss: 0.3181 - acc: 0.8728
Epoch 4/10
587/587 [==============================] - 9s 16ms/step - loss: 0.2786 - acc: 0.8955
Epoch 5/10
587/587 [==============================] - 9s 16ms/step - loss: 0.2545 - acc: 0.9053
Epoch 6/10
587/587 [==============================] - 9s 16ms/step - loss: 0.2364 - acc: 0.9167
Epoch 7/10
587/587 [==============================] - 9s 16ms/step - loss: 0.2205 - acc: 0.9229
Epoch 8/10
587/587 [==============================] - 9s 16ms/step - loss: 0.2081 - acc: 0.9292
Epoch 9/10
587/587 [==============================] - 9s 16ms/step - loss: 0.1971 - acc: 0.9352
Epoch 10/10
587/587 [==============================] - 9s 16ms/step - loss: 0.1868 - acc: 0.9375


In [20]:
def word2matrix(word):
    one_hots_word = []
    for ch in word:
        vec = np.zeros(vector_len)
        vec[char2idx[ch]] = 1
        one_hots_word.append(vec)
    
   
    for _ in range(max_word_len - len(word)):
        vec = np.zeros(vector_len)
        vec[char2idx['END']] = 1
        one_hots_word.append(vec)
        
    one_hots_word = np.array(one_hots_word)
    return one_hots_word

In [28]:
mat = word2matrix('naman')

In [29]:
mat.shape

(20, 39)

In [30]:
mat  = mat.reshape(1 , 20 , 39)

In [31]:
model.predict(mat)

array([[0.05675874, 0.6071494 ]], dtype=float32)

In [25]:
OneHotEncoder?

In [32]:
model.output.get_shape()

TensorShape([None, 2])